# Get Stock Data From Yahoo Finance

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import yfinance as yf
import os
# Progress bar
from tqdm.notebook import tqdm
# Stack trace
import traceback

# Stock data instructions
stock_dir = "stock_data"
ins_path = os.path.join(stock_dir, "stock_list.csv")
instructions = pd.read_csv(
    ins_path,
    index_col="ticker",
)
instructions

,start,end
ticker,,
AAPL,2020-03-09,2023-12-16
ACGLO,2023-11-28,2023-12-16
AMD,2016-12-12,2024-01-09
BABA,2014-11-07,2023-12-16
BHFAL,2023-11-30,2023-12-16
BPYPO,2023-11-30,2023-12-16
BROGW,2023-11-30,2023-12-16
CLSN,2009-09-28,2023-12-16
DIS,2019-06-13,2023-12-16


In [39]:
# Imports
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from curl_cffi import requests

# To avoid error
session = requests.Session(impersonate="chrome")

interval_len = 1 # year
interval_delay = .5 # seconds

def create_intervals(start, end):
    intervals = []
    current = start
    while current + relativedelta(years=interval_len) < end:
        next_time = current + relativedelta(years=interval_len)
        intervals.append((current, next_time))
        current = next_time
    intervals.append((current, end))  # final leftover chunk
    return intervals

def div_download(start, end):
    data = []

    for s, e in create_intervals(start, end):
        time.sleep(interval_delay)
        
        data.append(
            yf.download(
                ticker,
                start=s,
                end=e,
                interval="1d",
                actions=True,
                repair=True,
                session=session
            )
        )

    return pd.concat(data)

In [40]:
import traceback

failed_stocks = set()

# Get and store data from Yahoo! Finance
for ticker in tqdm(instructions.index, desc="Tickers Completed"):
    try:
        # Get price data
        start, end = instructions.loc[ticker, "start"], instructions.loc[ticker, "end"]
        fmt = "%Y-%m-%d"
        start, end = datetime.strptime(start, fmt), datetime.strptime(end, fmt)
        data = div_download(start, end)

        # Format data
        data.columns = data.columns.get_level_values("Price")
        data.columns.name = None
        data.columns = data.columns.to_list()
        data.drop(columns=["Repaired?"], inplace=True)
        data.rename(columns={label : label.lower() for label in data.columns})
        
        # Add to stocks directory
        path = os.path.join(stock_dir, f"{ticker}.csv")
        data.to_csv(path)
    except Exception:
        failed_stocks.add(ticker)
        print(f"Failed on stock {ticker}")
        traceback.print_exc()

Tickers Completed:   0%|          | 0/22 [00:00<?, ?it/s]

delayed


[*********************100%***********************]  1 of 1 completed


delayed


[*********************100%***********************]  1 of 1 completed


...

delayed


[*********************100%***********************]  1 of 1 completed


delayed


[*********************100%***********************]  1 of 1 completed

{'BROGW', 'CLSN'}


In [41]:
import pickle

# Useful ticker filter
useful = instructions.query("ticker not in @failed_stocks")

# Keep track of working stocks
with open("stocks.pkl", "wb") as downloaded:
    pickle.dump(useful, downloaded)

print(useful.shape[0])
useful

20


,start,end
ticker,,
AAPL,2020-03-09,2023-12-16
ACGLO,2023-11-28,2023-12-16
AMD,2016-12-12,2024-01-09
BABA,2014-11-07,2023-12-16
BHFAL,2023-11-30,2023-12-16
BPYPO,2023-11-30,2023-12-16
DIS,2019-06-13,2023-12-16
GILD,2009-09-11,2023-12-16
GOOG,2018-11-13,2023-12-16


In [42]:
print(failed_stocks)

{'BROGW', 'CLSN'}
